In [1]:
from transformers import pipeline 

phishing_detector = pipeline("text-classification", model="ealvaradob/bert-finetuned-phishing")


Device set to use cpu


In [2]:
import asyncio
import aioimaplib 
import email
from email.header import decode_header 

IMAP_SERVER = "imap.gmail.com"
EMAIL_USER = "ajbitheaddhz@gmail.com"
EMAIL_PASSWORD = "**Password**"

async def fetch_unread_emails():
    imap_client = aioimaplib.IMAP4_SSL(IMAP_SERVER)
    await imap_client.wait_hello_from_server()

    await imap_client.login(EMAIL_USER, EMAIL_PASSWORD)
    await imap_client.select("INBOX")

    _, email_ids = await imap_client.search("UNSEEN")
    emails = []
    for email_id in email_ids[0].split():
        _, data = await imap_client.fetch(email_id, "(RFC822)")
        raw_email = data[1][0][1]
        emails.append(raw_email)

    await imap_client.logout()
    return emails

### Process Emails Asynchronously


In [ ]:
async def process_email(raw_email):
    msg = email.message_from_bytes(raw_email)
    subject, encoding = decode_header(msg["Subject"])[0]
    subject = subject.decode(encoding or "utf-8") if isinstance(subject, bytes)

    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                body = part.get_payload(decode=True).decode()
                break
    else:
        body = msg.get_payload(decode=True).decode()

    prediction = phishing_detector(body)
    is_phishing = prediction[0]["label"] == "phishing"

    if is_phishing:
        print(f"! Phishing Email Detected! Subject: {subject}")

    return is_phishing

### Run Concurrently


In [ ]:
async def main():
    print("Fetching emails...")
    emails = await fetch_unread_emails()

    print(f"Processing {len(emails)} emails concurrently")
    tasks = [process_email(email) for email in emails]
    await asyncio.gather(*tasks)

    print("Phishing Detection completed.")

asyncio.run(main())